# Email Spam/Ham Classifier with PyTorch Neural Networks

Welcome! In this tutorial, you'll learn how to build a spam email classifier using deep learning.

## What You'll Learn

1. **Data Loading**: How to read and parse raw email files
2. **Text Preprocessing**: Cleaning and preparing text data for machine learning
3. **Feature Engineering**: Converting text to numbers (vectorization)
4. **Neural Networks**: Building a classifier with PyTorch
5. **Training**: How models learn from data
6. **Evaluation**: Measuring model performance

## Key Concepts Explained Along the Way

- What is a neural network?
- What are activation functions?
- What is overfitting and how to prevent it?
- Why dataset size matters for deep learning

Let's get started!

---
## Part 1: Setup and Imports

First, let's import all the libraries we'll need.

In [ ]:
# Standard library imports
import os
import re
import email
from email import policy
from pathlib import Path
from collections import Counter

# Data handling
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text processing
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# PyTorch for deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Progress bars
from tqdm.notebook import tqdm

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

---
## Part 2: Loading and Exploring the Data

### Understanding the Dataset

Our dataset contains raw email files organized into folders:
- `email-data-set/ham/hard_ham/` - Legitimate emails ("ham")
- `email-data-set/spam/spam/` - Spam emails

Each file is a complete email with headers (From, To, Subject, etc.) and body content.

### Why "Ham"?
In spam filtering, legitimate emails are called "ham" - the opposite of spam. It's a playful term used in the industry!

In [ ]:
# Define paths to our data
DATA_DIR = Path("email-data-set")
HAM_DIR = DATA_DIR / "ham" / "hard_ham"
SPAM_DIR = DATA_DIR / "spam" / "spam"

# Check if directories exist
print(f"Ham directory exists: {HAM_DIR.exists()}")
print(f"Spam directory exists: {SPAM_DIR.exists()}")

In [ ]:
def parse_email(file_path: Path) -> dict:
    """
    Parse an email file and extract useful information.
    
    Args:
        file_path: Path to the email file
        
    Returns:
        Dictionary with subject, from, body, and raw content
    """
    try:
        # Try different encodings (emails can be messy!)
        for encoding in ['utf-8', 'latin-1', 'ascii', 'cp1252']:
            try:
                with open(file_path, 'r', encoding=encoding, errors='ignore') as f:
                    raw_content = f.read()
                break
            except UnicodeDecodeError:
                continue
        
        # Parse the email using Python's email library
        msg = email.message_from_string(raw_content, policy=policy.default)
        
        # Extract subject
        subject = msg.get('Subject', '')
        if subject is None:
            subject = ''
        
        # Extract sender
        from_addr = msg.get('From', '')
        if from_addr is None:
            from_addr = ''
        
        # Extract body
        body = ''
        if msg.is_multipart():
            # Email has multiple parts (text, HTML, attachments)
            for part in msg.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':
                    try:
                        body = part.get_content()
                        break
                    except:
                        pass
                elif content_type == 'text/html' and not body:
                    try:
                        body = part.get_content()
                    except:
                        pass
        else:
            # Simple email with single body
            try:
                body = msg.get_content()
            except:
                body = raw_content
        
        return {
            'subject': str(subject),
            'from': str(from_addr),
            'body': str(body) if body else '',
            'raw': raw_content
        }
    except Exception as e:
        # If parsing fails, return raw content
        return {
            'subject': '',
            'from': '',
            'body': raw_content if 'raw_content' in dir() else '',
            'raw': raw_content if 'raw_content' in dir() else ''
        }

In [ ]:
def load_emails(directory: Path, label: int) -> list:
    """
    Load all emails from a directory.
    
    Args:
        directory: Path to directory containing email files
        label: 0 for ham, 1 for spam
        
    Returns:
        List of dictionaries with email data and labels
    """
    emails = []
    
    # Get all files in directory (exclude hidden files)
    files = [f for f in directory.iterdir() if f.is_file() and not f.name.startswith('.')]
    
    for file_path in tqdm(files, desc=f"Loading {'spam' if label else 'ham'}"):
        parsed = parse_email(file_path)
        parsed['label'] = label
        parsed['filename'] = file_path.name
        emails.append(parsed)
    
    return emails

# Load all emails
print("Loading emails...")
ham_emails = load_emails(HAM_DIR, label=0)  # 0 = ham
spam_emails = load_emails(SPAM_DIR, label=1)  # 1 = spam

# Combine into a single list
all_emails = ham_emails + spam_emails

print(f"\nLoaded {len(ham_emails)} ham emails")
print(f"Loaded {len(spam_emails)} spam emails")
print(f"Total: {len(all_emails)} emails")

In [ ]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(all_emails)

# Display basic info
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nLabel distribution:")
print(df['label'].value_counts().rename({0: 'Ham', 1: 'Spam'}))

### Visualizing the Dataset

Let's see the distribution of our data. Notice we have **class imbalance** - more spam than ham. This is common in real-world datasets and something we'll need to consider.

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart
labels = ['Ham (Legitimate)', 'Spam']
counts = [len(ham_emails), len(spam_emails)]
colors = ['#2ecc71', '#e74c3c']

axes[0].bar(labels, counts, color=colors)
axes[0].set_ylabel('Number of Emails')
axes[0].set_title('Email Distribution by Class')
for i, v in enumerate(counts):
    axes[0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Class Proportion')

plt.tight_layout()
plt.show()

In [ ]:
# Let's look at some example emails
print("=" * 60)
print("EXAMPLE HAM EMAIL")
print("=" * 60)
ham_example = df[df['label'] == 0].iloc[0]
print(f"Subject: {ham_example['subject'][:100]}")
print(f"From: {ham_example['from'][:50]}")
print(f"Body (first 500 chars):\n{ham_example['body'][:500]}")

print("\n" + "=" * 60)
print("EXAMPLE SPAM EMAIL")
print("=" * 60)
spam_example = df[df['label'] == 1].iloc[0]
print(f"Subject: {spam_example['subject'][:100]}")
print(f"From: {spam_example['from'][:50]}")
print(f"Body (first 500 chars):\n{spam_example['body'][:500]}")

---
## Part 3: Text Preprocessing

### Why Preprocess?

Raw text is messy! Emails contain:
- HTML tags
- Special characters
- URLs
- Email addresses
- Numbers
- Mixed capitalization

We need to **clean** the text so our model can focus on the meaningful content.

### Preprocessing Steps:
1. Remove HTML tags
2. Convert to lowercase
3. Remove URLs and email addresses
4. Remove special characters and numbers
5. Remove extra whitespace

In [ ]:
def clean_text(text: str) -> str:
    """
    Clean and preprocess email text.
    
    Args:
        text: Raw email text
        
    Returns:
        Cleaned text string
    """
    if not text or not isinstance(text, str):
        return ''
    
    # 1. Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator=' ')
    
    # 2. Convert to lowercase
    text = text.lower()
    
    # 3. Remove URLs
    text = re.sub(r'http\S+|www\.\S+', ' url ', text)
    
    # 4. Remove email addresses
    text = re.sub(r'\S+@\S+', ' emailaddr ', text)
    
    # 5. Remove numbers (but keep word tokens like 'free')
    text = re.sub(r'\b\d+\b', ' ', text)
    
    # 6. Remove special characters, keep only letters and spaces
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # 7. Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
# Test our cleaning function
sample_text = """
<html><body>
<h1>CONGRATULATIONS! You've WON $1,000,000!!!</h1>
<p>Click here: http://spam-link.com to claim your prize!</p>
<p>Contact: winner@spam.com for more info.</p>
</body></html>
"""

print("BEFORE cleaning:")
print(sample_text)
print("\nAFTER cleaning:")
print(clean_text(sample_text))

In [ ]:
# Apply cleaning to all emails
# We'll combine subject and body for classification
print("Cleaning all emails...")

def prepare_email_text(row):
    """Combine subject and body, then clean."""
    combined = f"{row['subject']} {row['body']}"
    return clean_text(combined)

df['clean_text'] = df.apply(prepare_email_text, axis=1)

# Check for empty texts
empty_count = (df['clean_text'].str.len() == 0).sum()
print(f"Emails with empty cleaned text: {empty_count}")

# Remove emails with empty text
df = df[df['clean_text'].str.len() > 0].reset_index(drop=True)
print(f"Remaining emails after removing empty: {len(df)}")

In [ ]:
# Analyze text lengths
df['text_length'] = df['clean_text'].str.len()
df['word_count'] = df['clean_text'].str.split().str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Text length distribution by class
for label, color, name in [(0, '#2ecc71', 'Ham'), (1, '#e74c3c', 'Spam')]:
    subset = df[df['label'] == label]['text_length']
    axes[0].hist(subset, bins=50, alpha=0.6, color=color, label=name)
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Text Length Distribution')
axes[0].legend()
axes[0].set_xlim(0, 5000)  # Limit x-axis for visibility

# Word count distribution
for label, color, name in [(0, '#2ecc71', 'Ham'), (1, '#e74c3c', 'Spam')]:
    subset = df[df['label'] == label]['word_count']
    axes[1].hist(subset, bins=50, alpha=0.6, color=color, label=name)
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Word Count Distribution')
axes[1].legend()
axes[1].set_xlim(0, 1000)

plt.tight_layout()
plt.show()

print("\nText statistics by class:")
print(df.groupby('label')[['text_length', 'word_count']].describe())

---
## Part 4: Feature Engineering - Converting Text to Numbers

### The Challenge

Neural networks work with **numbers**, not text. We need to convert our cleaned text into numerical vectors.

### TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF is a technique that converts text into numbers based on:

1. **Term Frequency (TF)**: How often a word appears in a document
2. **Inverse Document Frequency (IDF)**: How rare a word is across all documents

**Key Insight**: Words that appear frequently in one email but rarely in others are more important for classification.

For example:
- "the" appears everywhere → low importance
- "viagra" appears only in spam → high importance for spam detection

In [ ]:
# Split data into training and testing sets FIRST
# This is crucial to prevent data leakage!

X = df['clean_text'].values
y = df['label'].values

# Split: 80% training, 20% testing
# stratify=y ensures both sets have similar class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} emails")
print(f"Test set: {len(X_test)} emails")
print(f"\nTraining class distribution:")
print(f"  Ham: {sum(y_train == 0)} ({100*sum(y_train == 0)/len(y_train):.1f}%)")
print(f"  Spam: {sum(y_train == 1)} ({100*sum(y_train == 1)/len(y_train):.1f}%)")

In [ ]:
# Create TF-IDF vectorizer
# max_features limits vocabulary size (helps prevent overfitting)
# ngram_range=(1,2) includes single words and pairs of words

tfidf = TfidfVectorizer(
    max_features=5000,      # Use top 5000 most important words
    ngram_range=(1, 2),     # Include single words and word pairs
    min_df=2,               # Word must appear in at least 2 documents
    max_df=0.95,            # Ignore words in more than 95% of documents
    stop_words='english'    # Remove common English words (the, is, at, etc.)
)

# Fit on training data ONLY, then transform both
# This prevents "data leakage" from test set
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(f"Vocabulary size: {len(tfidf.vocabulary_)}")
print(f"Training matrix shape: {X_train_tfidf.shape}")
print(f"Test matrix shape: {X_test_tfidf.shape}")

In [ ]:
# Let's look at the most important words (features)
feature_names = tfidf.get_feature_names_out()

# Calculate average TF-IDF score for each class
ham_mask = y_train == 0
spam_mask = y_train == 1

ham_mean = np.array(X_train_tfidf[ham_mask].mean(axis=0)).flatten()
spam_mean = np.array(X_train_tfidf[spam_mask].mean(axis=0)).flatten()

# Find words most associated with each class
diff = spam_mean - ham_mean

# Top spam words (highest positive difference)
spam_indices = diff.argsort()[-15:][::-1]
print("Top words associated with SPAM:")
for idx in spam_indices:
    print(f"  {feature_names[idx]}: {diff[idx]:.4f}")

print("\nTop words associated with HAM:")
ham_indices = diff.argsort()[:15]
for idx in ham_indices:
    print(f"  {feature_names[idx]}: {diff[idx]:.4f}")

---
## Part 5: Building the Neural Network

### What is a Neural Network?

A neural network is a series of mathematical operations that learn patterns from data. Think of it like this:

```
Input (TF-IDF features) → Hidden Layers → Output (spam/ham probability)
```

### Key Components:

1. **Layers**: Groups of "neurons" that process information
2. **Weights**: Numbers that the network learns to adjust
3. **Activation Functions**: Add non-linearity (allow learning complex patterns)
4. **Dropout**: Randomly disable neurons during training (prevents overfitting)

### Our Architecture:

```
Input (5000 features)
    ↓
Dense Layer (256 neurons) + ReLU + Dropout
    ↓
Dense Layer (128 neurons) + ReLU + Dropout
    ↓
Dense Layer (64 neurons) + ReLU + Dropout
    ↓
Output Layer (1 neuron) + Sigmoid → Probability of Spam
```

In [ ]:
class SpamClassifier(nn.Module):
    """
    A feedforward neural network for spam classification.
    
    Architecture:
    - Input layer: accepts TF-IDF features
    - 3 hidden layers with ReLU activation and dropout
    - Output layer with sigmoid activation for binary classification
    """
    
    def __init__(self, input_dim: int, dropout_rate: float = 0.3):
        """
        Initialize the network.
        
        Args:
            input_dim: Number of input features (vocabulary size)
            dropout_rate: Probability of dropping neurons (0.3 = 30%)
        """
        super(SpamClassifier, self).__init__()
        
        # Define layers
        self.network = nn.Sequential(
            # First hidden layer
            nn.Linear(input_dim, 256),  # input_dim -> 256 neurons
            nn.ReLU(),                   # Activation function
            nn.Dropout(dropout_rate),    # Regularization
            
            # Second hidden layer
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Third hidden layer
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Output layer
            nn.Linear(64, 1),
            nn.Sigmoid()  # Output probability between 0 and 1
        )
    
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x: Input tensor of shape (batch_size, input_dim)
            
        Returns:
            Output tensor of shape (batch_size, 1) with spam probabilities
        """
        return self.network(x)

In [ ]:
# Create the model
input_dim = X_train_tfidf.shape[1]  # Number of TF-IDF features
model = SpamClassifier(input_dim=input_dim, dropout_rate=0.3)

# Move model to GPU if available
model = model.to(device)

# Print model architecture
print("Model Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

### Understanding Key Concepts

**ReLU (Rectified Linear Unit)**:
- Formula: `f(x) = max(0, x)`
- Simply: if input is negative, output is 0; otherwise, output equals input
- Why use it? Adds non-linearity so the network can learn complex patterns

**Dropout**:
- Randomly sets some neuron outputs to 0 during training
- Prevents the network from relying too much on specific neurons
- Acts as regularization to prevent overfitting

**Sigmoid**:
- Formula: `f(x) = 1 / (1 + e^(-x))`
- Squashes output to range [0, 1]
- Perfect for binary classification: output is probability of spam

---
## Part 6: Training the Model

### The Training Process

1. **Forward Pass**: Feed data through the network, get predictions
2. **Calculate Loss**: Measure how wrong the predictions are
3. **Backward Pass**: Calculate gradients (how to adjust weights)
4. **Update Weights**: Adjust weights to reduce loss
5. **Repeat** for many iterations (epochs)

### Key Training Components:

- **Loss Function**: Binary Cross-Entropy (measures prediction error)
- **Optimizer**: Adam (efficiently adjusts weights)
- **Batch Size**: Process 32 emails at a time
- **Epochs**: Complete passes through the training data

In [ ]:
# Convert sparse matrices to dense tensors
X_train_tensor = torch.FloatTensor(X_train_tfidf.toarray()).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_tfidf.toarray()).to(device)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1).to(device)

# Create DataLoaders for batching
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Define loss function and optimizer

# Binary Cross-Entropy Loss
# - Measures difference between predicted probability and true label
# - Lower is better
criterion = nn.BCELoss()

# Adam optimizer
# - lr (learning rate): how big steps to take when adjusting weights
# - weight_decay: L2 regularization to prevent overfitting
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    """
    Train for one epoch.
    
    Returns:
        Average training loss for the epoch
    """
    model.train()  # Set model to training mode (enables dropout)
    total_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Zero gradients from previous batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_X)
        
        # Calculate loss
        loss = criterion(outputs, batch_y)
        
        # Backward pass (calculate gradients)
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)


def evaluate(model, data_loader, criterion):
    """
    Evaluate model on a dataset.
    
    Returns:
        loss: Average loss
        accuracy: Classification accuracy
        predictions: Model predictions
        labels: True labels
    """
    model.eval()  # Set model to evaluation mode (disables dropout)
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for batch_X, batch_y in data_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
            
            # Convert probabilities to predictions (threshold = 0.5)
            preds = (outputs >= 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    return avg_loss, accuracy, np.array(all_preds), np.array(all_labels)

In [ ]:
# Training loop
num_epochs = 50
history = {
    'train_loss': [],
    'test_loss': [],
    'train_acc': [],
    'test_acc': []
}

# Early stopping: stop if test loss doesn't improve
best_test_loss = float('inf')
patience = 10
patience_counter = 0
best_model_state = None

print("Starting training...")
print("=" * 60)

for epoch in range(num_epochs):
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    
    # Evaluate on both sets
    _, train_acc, _, _ = evaluate(model, train_loader, criterion)
    test_loss, test_acc, _, _ = evaluate(model, test_loader, criterion)
    
    # Store history
    history['train_loss'].append(train_loss)
    history['test_loss'].append(test_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    # Print progress every 5 epochs
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
    
    # Early stopping check
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}!")
            break

# Restore best model
if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"\nRestored best model (test loss: {best_test_loss:.4f})")

print("=" * 60)
print("Training complete!")

In [ ]:
# Visualize training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Training Loss', color='blue')
axes[0].plot(history['test_loss'], label='Test Loss', color='red')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Test Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(history['train_acc'], label='Training Accuracy', color='blue')
axes[1].plot(history['test_acc'], label='Test Accuracy', color='red')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Test Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Understanding the Training Curves

**What to look for:**

1. **Both losses decreasing**: Model is learning!
2. **Train loss << Test loss (big gap)**: Overfitting - model memorizes training data
3. **Both losses similar**: Good generalization
4. **Losses plateau**: Model has learned what it can

**Overfitting** happens when:
- Training accuracy is high but test accuracy is lower
- The model memorizes training data instead of learning general patterns
- Common with small datasets (like ours!)

**Solutions we used:**
- Dropout layers
- Weight decay (L2 regularization)
- Early stopping

---
## Part 7: Evaluating the Model

### Classification Metrics

- **Accuracy**: % of correct predictions (can be misleading with imbalanced data)
- **Precision**: Of emails predicted as spam, how many were actually spam?
- **Recall**: Of actual spam emails, how many did we catch?
- **F1 Score**: Harmonic mean of precision and recall

### For Spam Detection:
- **High Precision** = Few false positives (legitimate emails marked as spam)
- **High Recall** = Catch most spam (few spam emails get through)

In [ ]:
# Final evaluation on test set
test_loss, test_acc, y_pred, y_true = evaluate(model, test_loader, criterion)

# Flatten predictions
y_pred = y_pred.flatten()
y_true = y_true.flatten()

print("=" * 60)
print("FINAL MODEL EVALUATION")
print("=" * 60)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

print("\n" + "=" * 60)
print("Classification Report:")
print("=" * 60)
print(classification_report(y_true, y_pred, target_names=['Ham', 'Spam']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Ham', 'Spam'],
            yticklabels=['Ham', 'Spam'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Interpret the confusion matrix
tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Interpretation:")
print(f"  True Negatives (Ham correctly identified): {tn}")
print(f"  False Positives (Ham incorrectly marked as Spam): {fp}")
print(f"  False Negatives (Spam that got through): {fn}")
print(f"  True Positives (Spam correctly caught): {tp}")

---
## Part 8: Making Predictions on New Emails

Let's see how to use our trained model to classify new emails.

In [ ]:
def predict_email(text: str, model, tfidf_vectorizer, device, threshold: float = 0.5):
    """
    Predict if an email is spam or ham.
    
    Args:
        text: Email text (subject + body)
        model: Trained PyTorch model
        tfidf_vectorizer: Fitted TF-IDF vectorizer
        device: torch device (cpu/cuda)
        threshold: Classification threshold (default 0.5)
        
    Returns:
        Dictionary with prediction and probability
    """
    # Clean the text
    cleaned = clean_text(text)
    
    # Vectorize
    features = tfidf_vectorizer.transform([cleaned]).toarray()
    features_tensor = torch.FloatTensor(features).to(device)
    
    # Predict
    model.eval()
    with torch.no_grad():
        probability = model(features_tensor).item()
    
    return {
        'prediction': 'SPAM' if probability >= threshold else 'HAM',
        'spam_probability': probability,
        'confidence': max(probability, 1 - probability)
    }

In [ ]:
# Test with some example emails

test_emails = [
    {
        'name': 'Obvious Spam',
        'text': '''
            CONGRATULATIONS! You have been selected to receive $1,000,000!
            Click here NOW to claim your prize! Limited time offer!
            ACT NOW! FREE MONEY! You're a winner!
        '''
    },
    {
        'name': 'Legitimate Work Email',
        'text': '''
            Hi team,
            
            Just a reminder that our meeting has been moved to 3pm tomorrow.
            Please review the attached agenda before the meeting.
            
            Thanks,
            John
        '''
    },
    {
        'name': 'Newsletter',
        'text': '''
            Your Weekly Tech Newsletter
            
            This week in tech: Apple announces new products, 
            Google updates search algorithm, and more.
            
            Click to read more articles on our website.
            Unsubscribe from this newsletter.
        '''
    },
    {
        'name': 'Phishing Attempt',
        'text': '''
            URGENT: Your account has been compromised!
            
            We detected suspicious activity on your account.
            Click here immediately to verify your identity and
            prevent your account from being suspended.
            
            Enter your password and credit card to confirm.
        '''
    }
]

print("=" * 60)
print("TESTING MODEL ON NEW EMAILS")
print("=" * 60)

for email_test in test_emails:
    result = predict_email(email_test['text'], model, tfidf, device)
    print(f"\n{email_test['name']}:")
    print(f"  Prediction: {result['prediction']}")
    print(f"  Spam Probability: {result['spam_probability']:.2%}")
    print(f"  Confidence: {result['confidence']:.2%}")

---
## Part 9: Saving the Model

Let's save our trained model and vectorizer so we can use them later.

In [ ]:
import pickle

# Create models directory
models_dir = Path('models')
models_dir.mkdir(exist_ok=True)

# Save PyTorch model
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': input_dim,
    'dropout_rate': 0.3
}, models_dir / 'spam_classifier.pth')

# Save TF-IDF vectorizer
with open(models_dir / 'tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("Model saved to 'models/' directory!")
print(f"  - spam_classifier.pth ({(models_dir / 'spam_classifier.pth').stat().st_size / 1024:.1f} KB)")
print(f"  - tfidf_vectorizer.pkl ({(models_dir / 'tfidf_vectorizer.pkl').stat().st_size / 1024:.1f} KB)")

---
## Summary and Key Takeaways

### What We Built
A neural network spam classifier using PyTorch that:
1. Loads and parses raw email files
2. Cleans and preprocesses text
3. Converts text to TF-IDF features
4. Classifies emails as spam or ham

### Key Concepts Learned

1. **Text Preprocessing**: Essential for NLP - clean, normalize, and tokenize text

2. **TF-IDF Vectorization**: Convert text to numbers while preserving importance

3. **Neural Network Architecture**:
   - Input layer matches feature dimensions
   - Hidden layers learn patterns
   - Activation functions add non-linearity
   - Output layer produces predictions

4. **Training Process**:
   - Forward pass → Loss calculation → Backward pass → Weight update
   - Monitor both training and validation metrics

5. **Regularization**:
   - Dropout prevents overfitting
   - Early stopping prevents overtraining

6. **Evaluation Metrics**:
   - Accuracy alone isn't enough for imbalanced datasets
   - Precision, recall, and F1 give fuller picture

### Limitations and Improvements

Our model has some limitations:
- **Small dataset** (752 emails): Deep learning typically needs more data
- **Simple architecture**: Could try LSTM or Transformers for better results
- **TF-IDF features**: Word embeddings (Word2Vec, BERT) could capture more meaning

### Next Steps

To improve, you could:
1. Collect more training data
2. Try pre-trained embeddings (GloVe, BERT)
3. Experiment with recurrent networks (LSTM, GRU)
4. Implement cross-validation for more robust evaluation
5. Add data augmentation techniques

Congratulations on building your first spam classifier!